# Lets try and save me some man hours lol 

for R? C:\Users\evana\AppData\Local\binman\binman_chromedriver\win32
for python? C:\Users\evana\AppData\Local\binman\binman_chromedriver\win32\129.0.6668.100



## Web Scraping

In [48]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import numpy as np
import pandas as pd
import time

In [ ]:
# A new approach 
# Web scrape under property information until there is no longer any elements, then web scrape under property info
#if price len == property info len, start using the property info for titles 

In [ ]:
#Functionize this to include property info as a parameter.
price = []
price_parents = "property-info"
price_names = ["property-pricing", "property-rents", "price-range"]

def source_class(web_driver, class_parents, class_list, class_names):
    listings = []
    if isinstance(class_parents, list): # multiple parents
        for parent in class_parents: # joins all elements that fall under parent nodes 
            listings += web_driver.find_elements(By.CLASS_NAME, parent)
    else: # just 1 possible parent
        listings = web_driver.find_elements(By.CLASS_NAME, class_parents)
    for listing in listings: # *** LOGIC ERROR*** In this approach the title & address will have more listings, so it will return None more times, 
        found_flag = False
        for class_name in class_names: # will check every class name within each parent 
            if len(listing.find_elements(By.CLASS_NAME, class_name)) != 0: # found for current class
                class_list.append(listing.find_element(By.CLASS_NAME, class_name).text)
                found_flag = True
                break
            else: # not found
                continue # try next name
        if not found_flag: # never found
            class_list.append(None)
    return



In [ ]:
# Lets correct this function:

def source_class(web_driver, class_parents, class_list, class_names):
    listings = []
    if isinstance(class_parents, list): # multiple parents
        for parent in class_parents: # joins all elements that fall under parent nodes 
            listings = web_driver.find_elements(By.CLASS_NAME, parent)
            found_flag = False
            print(f"Checking parent: {parent}")
            for listing in listings: 
                for class_name in class_names:
                    if len(listing.find_elements(By.CLASS_NAME, class_name)) != 0: # found for current class
                        class_list.append(listing.find_element(By.CLASS_NAME, class_name).text)
                        found_flag = True
                        print(f"Class found under: {parent}: {class_name}")
                        break
                    else: # not found
                        # print(f"Class not found for: {parent}: {class_name}")
                        continue # try next name
                if not found_flag and parent == class_parents[len(class_parents) - 1]: # never found & we are on the last parent
                    print(f"Class not found in any parent, Appending None")
                    class_list.append(None)
                    return
                else:
                    print(f"Element not found for parent: {parent}")
                    break
            
            
    else: # just 1 possible parent
        listings = web_driver.find_elements(By.CLASS_NAME, class_parents)
    for listing in listings: # *** LOGIC ERROR*** In this approach the title & address will have more listings, so it will return None more times, 
        found_flag = False
        for class_name in class_names: # will check every class name within each parent 
            if len(listing.find_elements(By.CLASS_NAME, class_name)) != 0: # found for current class
                class_list.append(listing.find_element(By.CLASS_NAME, class_name).text)
                found_flag = True
                break
            else: # not found
                continue # try next name
        if not found_flag: # never found
            class_list.append(None)
    return



In [ ]:
# The god cell

# Get us to website
web = "https://www.apartments.com/off-campus-housing/ca/los-angeles/university-of-california-los-angeles/"
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options) 
driver.get(web)

#Prepare data 
title_parents = ["property-information", "property-info"]
address_parents = ["property-information", "property-info"]
price_parents = ["property-info"]
bed_parents = ["property-info"]
extra_parents = ["property-info"]

titles, addresses, prices, beds, ammenities = [], [], [], [], []

price_names = ["property-pricing", "property-rents", "price-range"]
bed_names = ["property-beds", "bed-range"]
title_names = ['property-title', 'js-placardTitle']
address_names =["property-address"]
extra_names = ["property-amenities"]

#Begin the scraping
page = 1
while(True):
    time.sleep(3)
    print(f"Current page is {page}")
   
    source_class(driver, title_parents, titles, title_names)
    source_class(driver, address_parents, addresses, address_names)
    source_class(driver, price_parents, prices, price_names)
    source_class(driver, bed_parents, beds, bed_names)
    source_class(driver, extra_parents, ammenities, extra_names)
    
    #Cylce to next page
    button_list = driver.find_elements(By.CLASS_NAME, "pagingBtn")
    next = button_list[len(button_list) - 1]
    if next.text == "Next":
        page+=1
        next.click()
    else:
        break
    
driver.close()

Current page is 1
Current page is 2
Current page is 3
Current page is 4
Current page is 5
Current page is 6
Current page is 7
Current page is 8
Current page is 9
Current page is 10
Current page is 11
Current page is 12
Current page is 13
Current page is 14
Current page is 15
Current page is 16
Current page is 17
Current page is 18


In [52]:
data = {
    "Title": titles[:700],
    "Address": addresses[:700],
    "Price": prices,
    "Beds": beds,
    "Ammenities": ammenities,
}

table = pd.DataFrame(data)


In [58]:
table[table.Address.str.contains("Federal", case=False, na=False)]

,Title,Address,Price,Beds,Ammenities
0,1223 Federal Ave,"1223 Federal Ave, Los Angeles, CA 90025","$2,698 - $2,998",2 Beds,Pets Allowed Pool
80,Federal Apartments,"1814 Federal Ave, Los Angeles, CA 90025","$2,700 - $4,975",1-2 Beds,None
335,1517 Federal Ave,"1517 Federal Ave, Los Angeles, CA 90025","$2,195","1 Bed, 1 Bath, 617 sq ft",None
341,1650 Federal Ave,"1650 Federal Ave, Los Angeles, CA 90025","$4,410","1 Bed, 1.5 Baths, 750 sq ft",None


In [ ]:
# import pickle

# with open('table.pkl', 'wb') as f:
#     pickle.dump(table, f)
# So we dont have to keep rerunning the webscraper



In [38]:
web = "https://www.apartments.com/off-campus-housing/ca/los-angeles/university-of-california-los-angeles/4/"
# path = '/Users/evana/AppData/Local/binman/binman_chromedriver/win32/129.0.6668.100'
# service = Service(executable_path=path)

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

driver.get(web)

# Going foreard, try the old titles and if they fail try the new ttitles and if that fails then append None lol 


In [37]:
driver.close()

In [39]:
titles = []
title_parents = ["property-information", "property-info"]
title_names = ["js-placardTitle", "property-title"]

source_class(driver, title_parents, titles, title_names)

print(titles)

['462 S Beverwil Dr', '2852 Sawtelle Blvd Unit 33', '605 Kelton Ave Unit FL2-ID1277', '451 S Barrington Ave Unit 301', '11714 Goshen Ave', '1850 Camden Ave Unit #7', '1945 Glendon Ave Unit FL1-ID1279', '1416 Barry Ave', '1652 Colby Ave Unit 102', '1813 Barry Ave', '1437 Brockton Ave Unit 05', '12124 Goshen Ave Unit 106', '2435 S Sepulveda Blvd Unit FL7-ID1048', '2302 32nd St Unit C', None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'Kelton Towers Apartments', '1400 Camden Ave', '1626 Malcolm Ave', '11515 Rochester Ave', 'Selby Venti', '1750 S Bentley Ave', '1906 Pelham Ave', '1650 Sawtelle Blvd', 'Empire at Bellagio', '1771 Beloit Ave', '1806 CORINTH AV', '1837 Sawtelle Blvd', 'Ohio Rose Apartments', '1015 S Westgate Ave Unit 3', '11900 Montana Ave', '1440 Armacost Ave', '10340-10366 Bellwood Ave', '1222 Amherst - fully renovated unit in Los...', '1525 S Saltair Ave', '1550 Saltair Ave, prime West LA location!', 'Pico Federal Apartments', 'Sepulveda C

In [ ]:
#Functionize this to include property info as a parameter.
price = []
price_names = ["property-pricing", "property-rents", "price-range"]
found_flag = False
title_addr = driver.find_elements(By.CLASS_NAME, "property-info")
for listing in title_addr:
    for class_name in price_names:
        if len(listing.find_elements(By.CLASS_NAME, class_name)) != 0:
            price.append(listing.find_element(By.CLASS_NAME, class_name).text)
            found_flag = True
            break # will be return # but why? shouldn't it move on to the next listng, thats what happens 
        else: # not found
            continue # try next name
    if not found_flag:
        price.append(None)
print(price)


['$2,895', '$4,500', '$6,100 - $7,295', '$2,675', '$3,300', '$2,295', '$2,895 - $3,295', '$3,395 - $3,595', '$3,345', '$2,895 - $3,195', '$2,000 - $2,400', '$2,295', '$1,925', '$3,575 - $3,650', '$3,095 - $4,995', '$2,350', '$2,150 - $3,995', '$1,995', '$1,800', '$3,695', '$2,295 - $5,899', 'Call for Rent', '$2,950 - $2,995', '$2,495 - $4,995', '$3,100 - $4,000', '$4,295', 'Call for Rent', '$2,395 - $2,495', '$3,200', '$3,395', '$3,195 - $5,195', '$2,495', '$2,795', '$2,500', '$3,895 - $4,750', '$3,600', '$2,895 - $4,495', '$2,300', '$1,550 - $1,650', '$4,300']


In [9]:
zero_list = []
first_list = [1,2,3]
second_list = [4,5,6]
first_second = [first_list, second_list]

print(zero_list)

for my_list in first_second:
    zero_list +=  my_list
    
# zero_list = [zero_list + my_list for my_list in first_second]
# zero_list = [item for my_list in first_second for item in my_list] # wow nested loop comprehension lol 


print(zero_list)

[]
[1, 2, 3, 4, 5, 6]


In [7]:
# Assume this function is run when were looking at a given overlaying class, so we should run it for
# property-information: title & address 
# property-info: everything else
# but if we obtain a title or address from property information, we don't want to run that command for title or addres in property info, make a flag for it 
def source_class_v2(listing, class_list, class_names, is_last_call = False):
    found_flag = False
    for class_name in class_names:
        if len(listing.find_elements(By.CLASS_NAME, class_name)) != 0: # found for current class
            class_list.append(listing.find_element(By.CLASS_NAME, class_name).text)
            found_flag = True
            return
        else: # not found
            continue # try next name
    if not found_flag and is_last_call: # never found & last call
        class_list.append(None)
    elif not found_flag: # never found
        return
    return
    # This function should take a list of class values & their possible class names, & it should append any values found with those names to the list. 


In [24]:
def source_class(listing, class_list, class_names):
    found_flag = False
    for class_name in class_names:
        # print(f"Checking {class_name} class")
        if len(listing.find_elements(By.CLASS_NAME, class_name)) != 0: # found for current class
            # print(f"Found {class_name} class")
            class_list.append(listing.find_element(By.CLASS_NAME, class_name).text)
            found_flag = True
            return
        else: # not found
            print(f"Class {class_name} not found, checking next")
            continue # try next name
    if not found_flag: # never found & last call
        print(f"Appending None for {listing}")
        class_list.append(None)
        return
    return
    # This function should take a list of class values & their possible class names, & it should append any values found with those names to the list. 


In [ ]:
### WORKING ON THIS ONE # DIDNT WORK

web = "https://www.apartments.com/off-campus-housing/ca/los-angeles/university-of-california-los-angeles/4/"
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options) 
driver.get(web)

title_addr = driver.find_elements(By.CLASS_NAME, "property-information") # becomes useless after a few pages 
room_info = driver.find_elements(By.CLASS_NAME, "property-info") # aggregate listing info 
title_found, address_found = False, False
titles = []
title_names = ['property-title', 'js-placardTitle']
# must incorporate the loop & greater header in order to generalize better,
# then finding the title will actually be true, won't have to worry about adding it agian 


addresses = []
address_names =["property-address"]


prices = []
price_names = ["property-pricing", "property-rents", "price-range"]

beds = []
bed_names = ["property-beds", "bed-range"]

ammenities = []
extra_names = ["property-amenities"]

time.sleep(3)

for listing in title_addr:
    source_class(listing,titles,title_names)
    source_class(listing,addresses, address_names)
    if titles[-1] == "2302 32nd St Unit C":
        print("STOPPING OLD HEADER")
        break
    # Find address, if so title found = True 
for listing in room_info:
    source_class(listing, prices, price_names)
    source_class(listing, beds, bed_names)
    source_class(listing, ammenities, extra_names)
    
driver.close()

#If title not found, search here 
#If address not found, search here 
    

In [28]:
prices

['$2,900',
 '$2,750',
 '$2,660',
 '$4,200',
 '$4,250',
 '$3,995',
 '$2,470',
 '$3,100 - $3,200',
 '$2,800',
 '$5,250',
 '$2,750',
 '$4,800']

In [ ]:
#Prepare data 
title_parents = ["property-information", "property-info"]
titles = []
title_names = ['property-title', 'js-placardTitle']

time.sleep(3)


for parent in title_parents:
    listings = driver.find_elements(By.CLASS_NAME, parent)
    last_call = [True if parent == title_parents[-1] else False]
    for listing in listings:
        source_class(listing, titles, title_names, last_call)

driver.close()

print(titles)


In [9]:
web = "https://www.apartments.com/off-campus-housing/ca/los-angeles/university-of-california-los-angeles/"
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options) 
driver.get(web)

In [ ]:
#Functionize this to include property info as a parameter.
price = []
price_names = ["property-pricing", "property-rents", "price-range"]
title_addr = driver.find_elements(By.CLASS_NAME, "property-information")
rest = driver.find_elements(By.CLASS_NAME, "property-info")
# Assuming a list
#case of title_addr:
last_call = False # since proprty information != the last element
for listing in title_addr: 
    source_class(price, price_names, last_call)
#case of rest 
last_call = True  # since property information IS the last element 
for listing in title_addr: 
    source_class(price, price_names, last_call) # will now append None if not found 
print(price)


[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [79]:
mylist = [1,2,3,4]
mylist[-1]

4

In [ ]:
# Get us to website
web = "https://www.apartments.com/off-campus-housing/ca/los-angeles/university-of-california-los-angeles/4/"
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options) 
driver.get(web)

#Prepare data 
title_parents = ["property-information", "property-info"]
titles = []
title_names = ['property-title', 'js-placardTitle']


time.sleep(3)

for parent in title_parents:
    listings = driver.find_elements(By.CLASS_NAME, parent)
    last_call = [True if parent == title_parents[-1] else False]
    for listing in listings:
        source_class(listing, titles, title_names, last_call)

driver.close()

print(titles)


['462 S Beverwil Dr', '2852 Sawtelle Blvd Unit 33', '605 Kelton Ave Unit FL2-ID1277', '451 S Barrington Ave Unit 301', '11714 Goshen Ave', '1850 Camden Ave Unit #7', '1945 Glendon Ave Unit FL1-ID1279', '1416 Barry Ave', '1652 Colby Ave Unit 102', '1813 Barry Ave', '1437 Brockton Ave Unit 05', '12124 Goshen Ave Unit 106', '2435 S Sepulveda Blvd Unit FL7-ID1048', '2302 32nd St Unit C', None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'Kelton Towers Apartments', '1400 Camden Ave', '1626 Malcolm Ave', '11515 Rochester Ave', 'Selby Venti', '1750 S Bentley Ave', '1906 Pelham Ave', '1650 Sawtelle Blvd', 'Empire at Bellagio', '1771 Beloit Ave', '1806 CORINTH AV', '1837 Sawtelle Blvd', 'Ohio Rose Apartments', '1015 S Westgate Ave Unit 3', '11900 Montana Ave', '1440 Armacost Ave', '10340-10366 Bellwood Ave', '1222 Amherst - fully renovated unit in Los...', '1525 S Saltair Ave', '1550 Saltair Ave, prime West LA location!', 'Pico Federal Apartments', 'Sepulveda C

In [ ]:
for parent in title_parents:
    listings = driver.find_elements(By.CLASS_NAME, parent)
    last_call = [True if parent == title_parents[-1] else False]
    for listing in listings:
        print("look at listing")

In [ ]:
pd.DataFrame(mini_data)

In [ ]:
my_dct = {"title":titles, "address": addresses}
pd.DataFrame(my_dct)


# Begin here anew

In [ ]:
#if title = "2302 32nd St Unit C", stop searching title_address_parents, begin searching rest alongside 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import numpy as np
import pandas as pd
import time

In [29]:
def remove_shared_none(lst1, lst2):
    # Using list comprehension with zip to filter out shared None locations
    filtered_lst1, filtered_lst2 = zip(*[(a, b) for a, b in zip(lst1, lst2) if not (a is None and b is None)])
    
    # Convert back to lists since zip returns tuples
    return list(filtered_lst1), list(filtered_lst2)


In [34]:
# Get us to website
web = "https://www.apartments.com/off-campus-housing/ca/los-angeles/university-of-california-los-angeles/"
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options) 
driver.get(web)

#Prepare data 
title_address_parents = ["property-information", "property-info"]

titles = []
title_names = ['property-title', 'js-placardTitle']

addresses = []
address_names =["property-address"]

prices = []
price_names = ["property-pricing", "property-rents", "price-range"]

beds = []
bed_names = ["property-beds", "bed-range"]

ammenities = []
extra_names = ["property-amenities"]

page = 1
while(True):
    
    print(f"Current page is {page}")
    time.sleep(2)
    for parent in title_address_parents:
        listings = driver.find_elements(By.CLASS_NAME, parent)
        last_call = (parent == title_address_parents[-1])
        for listing in listings:
            source_class_v2(listing, titles, title_names, last_call)
            source_class_v2(listing, addresses, address_names, last_call)
            
    rest_of_them = driver.find_elements(By.CLASS_NAME, "property-info")
    for listing in rest_of_them:
        source_class_v2(listing, prices, price_names, True)
        source_class_v2(listing, beds, bed_names, True)
        source_class_v2(listing, ammenities, extra_names, True)
            
    titles, addresses = remove_shared_none(titles, addresses)

    #Cylce to next page
    button_list = driver.find_elements(By.CLASS_NAME, "pagingBtn")
    next = button_list[len(button_list) - 1]
    if next.text == "Next":
        page+=1
        next.click()
    else:
        break
    
driver.close()


data = {
    "Title": titles,
    "Address": addresses,
    "Price": prices,
    "Beds": beds,
    "Ammenities": ammenities,
}
        

Current page is 1
Current page is 2
Current page is 3
Current page is 4
Current page is 5
Current page is 6
Current page is 7
Current page is 8
Current page is 9
Current page is 10
Current page is 11
Current page is 12
Current page is 13
Current page is 14
Current page is 15
Current page is 16
Current page is 17
Current page is 18


In [35]:
data = {
    "Title": titles,
    "Address": addresses,
    "Price": prices,
    "Beds": beds,
    "Ammenities": ammenities,
}

table = pd.DataFrame(data)

In [40]:
table.to_csv('apts.csv', index=False) 

In [41]:
table.sample(38)

,Title,Address,Price,Beds,Ammenities
140,1906 Pelham Ave,"1906 Pelham Ave, Los Angeles, CA 90025","$2,400 - $2,500",1 Bed,None
57,Midvale Apartments,"527-540 Midvale Ave, Los Angeles, CA 90024","$4,605 - $4,770",Studio - 2 Beds,None
589,"10517 Blythe Ave, Los Angeles, CA 90064",None,"$22,500","6 Beds, 6.5 Baths, 6,000 sq ft",None
178,1824 Greenfield Ave,"1824 Greenfield Ave, Los Angeles, CA 90025","$3,000",2 Beds,None
123,451 S Barrington Ave Unit 301,"451 S Barrington Ave Unit 301, Los Angeles, CA...","$4,200","2 Beds, 2 Baths, 1,460 sq ft",None
502,"606 Whittier Dr, Beverly Hills, CA 90210",None,"$14,900","4 Beds, 4 Baths, 4,287 sq ft",None
461,"100 S Mapleton Dr, Los Angeles, CA 90024",None,"$85,000","5 Beds, 8 Baths, 8,224 sq ft",None
612,"714 N Rexford Dr, Beverly Hills, CA 90210",None,"$29,900","5 Beds, 7 Baths, 5,995 sq ft",None
525,"1130 Benedict Canyon Drive, Beverly Hills, CA ...",None,"$19,950","4 Beds, 4 Baths, 4,291 sq ft",None
237,468 South Roxbury,"468 S Roxbury Dr, Beverly Hills, CA 90212","$3,895 - $4,750",2 Beds,None


In [33]:
my_dct = {"title" : titles, "address" : addresses}
pd.DataFrame(my_dct)

,title,address
0,462 S Beverwil Dr,"462 S Beverwil Dr, Beverly Hills, CA 90212"
1,2852 Sawtelle Blvd Unit 33,"2852 Sawtelle Blvd Unit 33, Los Angeles, CA 90064"
2,605 Kelton Ave Unit FL2-ID1277,"605 Kelton Ave Unit FL2-ID1277, Los Angeles, C..."
3,451 S Barrington Ave Unit 301,"451 S Barrington Ave Unit 301, Los Angeles, CA..."
4,11714 Goshen Ave,"11714 Goshen Ave, Los Angeles, CA 90049"
5,1850 Camden Ave Unit #7,"1850 Camden Ave Unit #7, Los Angeles, CA 90025"
6,1945 Glendon Ave Unit FL1-ID1279,"1945 Glendon Ave Unit FL1-ID1279, Los Angeles,..."
7,1416 Barry Ave,"1416 Barry Ave, Los Angeles, CA 90025"
8,1652 Colby Ave Unit 102,"1652 Colby Ave Unit 102, Los Angeles, CA 90025"
9,1813 Barry Ave,"1813 Barry Ave, Los Angeles, CA 90025"


In [ ]:
table.iloc[:,2:4].dropna(how = "all").reset_index().iloc[:,1:]

In [ ]:
table.iloc[:,:2].dropna(how = "all").reset_index().iloc[:,1:]

In [ ]:
# Get us to website
web = "https://www.apartments.com/off-campus-housing/ca/los-angeles/university-of-california-los-angeles/4/"
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options) 
driver.get(web)



driver.close()

print(addresses)


['462 S Beverwil Dr, Beverly Hills, CA 90212', '2852 Sawtelle Blvd Unit 33, Los Angeles, CA 90064', '605 Kelton Ave Unit FL2-ID1277, Los Angeles, CA 90024', '451 S Barrington Ave Unit 301, Los Angeles, CA 90049', '11714 Goshen Ave, Los Angeles, CA 90049', '1850 Camden Ave Unit #7, Los Angeles, CA 90025', '1945 Glendon Ave Unit FL1-ID1279, Los Angeles, CA 90025', '1416 Barry Ave, Los Angeles, CA 90025', '1652 Colby Ave Unit 102, Los Angeles, CA 90025', '1813 Barry Ave, Los Angeles, CA 90025', '1437 Brockton Ave Unit 05, Los Angeles, CA 90025', '12124 Goshen Ave Unit 106, Los Angeles, CA 90049', '2435 S Sepulveda Blvd Unit FL7-ID1048, Los Angeles, CA 90064', '2302 32nd St Unit C, Santa Monica, CA 90405', None, None, None, None, None, None, None, None, None, None, None, None, None, None, '515 Kelton Ave, Los Angeles, CA 90024', '1400 Camden Ave, Los Angeles, CA 90025', '1626 Malcolm Ave, Los Angeles, CA 90024', '11515 Rochester Ave, Los Angeles, CA 90025', '1830 Selby Ave, Los Angeles, CA

In [ ]:
if isinstance(class_parents, list): # multiple parents
    for parent in class_parents: # joins all elements that fall under parent nodes 
        listings = web_driver.find_elements(By.CLASS_NAME, parent)
        print(f"Checking parent: {parent}")
        #If it is the last parent, last call = true 
        for listing in listings: 
            source_class(price, price_names, last_call) # will append None only when weve checked both parents
            
            
    else: # just 1 possible parent
        listings = web_driver.find_elements(By.CLASS_NAME, class_parents)
    for listing in listings: # *** LOGIC ERROR*** In this approach the title & address will have more listings, so it will return None more times, 
        found_flag = False
        for class_name in class_names: # will check every class name within each parent 
            if len(listing.find_elements(By.CLASS_NAME, class_name)) != 0: # found for current class
                class_list.append(listing.find_element(By.CLASS_NAME, class_name).text)
                found_flag = True
                break
            else: # not found
                continue # try next name
        if not found_flag: # never found
            class_list.append(None)
    return

In [25]:
title_addr = driver.find_elements(By.CLASS_NAME, "property-information")
rest_of_them = driver.find_elements(By.CLASS_NAME, "property-info")
print(title_addr + rest_of_them)


# for listing in title_addr:
#     print(f"Beds: {listing.find_elements(By.CLASS_NAME, "property-beds")}") # empty list element when using find_elements, doesnt cause error

        


[<selenium.webdriver.remote.webelement.WebElement (session="1dfb32447c1631e968af536e2f2b8097", element="f.9E495CF15CD8C471806E0199DE47B907.d.9B4A0639B93B69B31A806A4494FEA67F.e.1859")>, <selenium.webdriver.remote.webelement.WebElement (session="1dfb32447c1631e968af536e2f2b8097", element="f.9E495CF15CD8C471806E0199DE47B907.d.9B4A0639B93B69B31A806A4494FEA67F.e.1892")>, <selenium.webdriver.remote.webelement.WebElement (session="1dfb32447c1631e968af536e2f2b8097", element="f.9E495CF15CD8C471806E0199DE47B907.d.9B4A0639B93B69B31A806A4494FEA67F.e.1925")>, <selenium.webdriver.remote.webelement.WebElement (session="1dfb32447c1631e968af536e2f2b8097", element="f.9E495CF15CD8C471806E0199DE47B907.d.9B4A0639B93B69B31A806A4494FEA67F.e.1960")>, <selenium.webdriver.remote.webelement.WebElement (session="1dfb32447c1631e968af536e2f2b8097", element="f.9E495CF15CD8C471806E0199DE47B907.d.9B4A0639B93B69B31A806A4494FEA67F.e.1995")>, <selenium.webdriver.remote.webelement.WebElement (session="1dfb32447c1631e968af

In [ ]:
# classes for property-info
# Price:[property-pricing, property-rents, price-range]
# Beds: [property-beds, bed-range]
# Title:[js-placardTitle]
# Address: [property-address]
# Ammentities: [property-amenities]
# Type: [property-type-for-rent]

# classes for property-information
# Title:[property-title]
# Address: [property-address]


In [34]:
title_addr = driver.find_elements(By.CLASS_NAME, "property-info")
print(len(title_addr))

titles = []
prices = []
for listing in title_addr:
    try:
        title = listing.find_element(By.CLASS_NAME, "js-placardTitle").text
        titles.append(title)
    except:
        titles.append(None)
        
    try:
        price = listing.find_element(By.CLASS_NAME, "price-range").text
        prices.append(price)
    except:
        prices.append(None)
        
        
print(titles, prices)

20
['616 N Maple Dr, Beverly Hills, CA 90210', '1634 N Beverly Dr, Beverly Hills, CA 90210', '2717 Motor Ave, Los Angeles, CA 90064', '1957 Linda Flora Dr, Los Angeles, CA 90077', '2593 S Westgate Ave, Los Angeles, CA 90064', '9372 W Olympic Blvd, Beverly Hills, CA 90212', '133 N Rexford Dr, Beverly Hills, CA 90210', '510 Stonewood Dr, Beverly Hills, CA 90210', '10304 Glenbarr Ave, Los Angeles, CA 90064', '1750 N Beverly Dr, Beverly Hills, CA 90210', '520 Stonewood Dr, Beverly Hills, CA 90210', '1251 Lago Vista Dr, Beverly Hills, CA 90210', '12019 Ocean Park Blvd Unit 12019.5, Los Angeles, CA 90064', '9958 Westwanda Dr, Beverly Hills, CA 90210', '748 24th St, Santa Monica, CA 90402', '611 N Hillcrest Rd, Beverly Hills, CA 90210', '9814 Curwood Pl, Beverly Hills, CA 90210', '2415 Arizona Ave Unit 2415, Santa Monica, CA 90404', '701 N Hillcrest Rd, Beverly Hills, CA 90210', '13324 Sunset Blvd, Los Angeles, CA 90049'] ['$40,000', '$10,500', '$27,500', '$16,000', '$3,450', '$3,700', '$9,00

In [83]:
driver.close()

In [ ]:
# The god cell

# Get us to website
web = "https://www.apartments.com/off-campus-housing/ca/los-angeles/university-of-california-los-angeles/"
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options) 
driver.get(web)

#Prepare data 
titles, addresses, prices, beds, ammenities = [], [], [], [], []

#Begin the scraping
page = 1
while(True):
    time.sleep(0.5)
    # On Page, add the info to each list 
    title_addr = driver.find_elements(By.CLASS_NAME, "property-information") # becomes useless after a few pages 
    room_info = driver.find_elements(By.CLASS_NAME, "property-info") # aggregate listing info 
    title_found, address_found = False, False
    # must incorporate the loop & greater header in order to generalize better,
    # then finding the title will actually be true, won't have to worry about adding it agian 
    
    for listing in title_addr:
        # Find title, if so title found = True 
        try:
        # Find address, if so title found = True 

    for listing in room_info:
        # Find Price
        # Find Beds
        # Find Ammenities
        
        #If title not found, search here 
        #If address not found, search here 
    
    #Cylce to next page
    button_list = driver.find_elements(By.CLASS_NAME, "pagingBtn")
    next = button_list[len(button_list) - 1]
    if next.text == "Next":
        page+=1
        print(f"Current page is {page}")
        next.click()
    else:
        break
    
driver.close()

# Issue is there is no title/address after the first 128 listings
# data = {
#     "Title": titles,
#     "Address": addresses,
#     "Price": prices,
#     "Beds": beds,
#     "Ammenities": ammenities,
# }

# table = pd.DataFrame(data)

Current page is 2
Current page is 3
Current page is 4
Current page is 5
Current page is 6
Current page is 7
Current page is 8
Current page is 9
Current page is 10
Current page is 11
Current page is 12
Current page is 13
Current page is 14
Current page is 15
Current page is 16
Current page is 17
Current page is 18


In [17]:
small_data = { "Title" : titles, "Address" : addresses}
small_table = pd.DataFrame(small_data)
small_table.info()
small_table.sample(8)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Title    133 non-null    object
 1   Address  133 non-null    object
dtypes: object(2)
memory usage: 2.2+ KB


,Title,Address
123,1850 Camden Ave Unit #7,"1850 Camden Ave Unit #7, Los Angeles, CA 90025"
3,2245 Beverly Glen Blvd.,"2245 S Beverly Glen Blvd, Los Angeles, CA 90064"
54,The Stanford West,"3008 Santa Monica Blvd, Santa Monica, CA 90404"
27,The SW by CLG,"1947 Sawtelle Blvd, Los Angeles, CA 90025"
12,The Glendon Apartments in Westwood,"1040 Glendon Ave, Los Angeles, CA 90024"
112,1712 Barry Ave Unit 1,"1712 Barry Ave Unit 1, Los Angeles, CA 90025"
16,The Plaza Apartments,"10983-10983 Wellworth Ave, Los Angeles, CA 90024"
5,Evani 3223 Wilshire,"3223 Wilshire Blvd, Santa Monica, CA 90403"


In [25]:
medium_data = { "Prices" : prices, "Beds" : beds, "Extras": ammenities}
medium_table = pd.DataFrame(medium_data)
medium_table.info()
medium_table

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Prices  133 non-null    object
 1   Beds    250 non-null    object
 2   Extras  36 non-null     object
dtypes: object(3)
memory usage: 16.5+ KB


,Prices,Beds,Extras
0,"$3,880 - $11,750",Studio - 2 Beds,Pets Allowed Fitness Center Pool In Unit Washe...
1,"$3,055 - $425,500",1-2 Beds,Pets Allowed Fitness Center Pool
2,"$10,915 - $65,000",1-4 Beds,Pets Allowed Fitness Center Pool
3,"$2,298 - $3,348",Studio - 2 Beds,None
4,"$3,145 - $11,215",Studio - 3 Beds,Pets Allowed Fitness Center Pool
...,...,...,...
695,None,None,None
696,None,None,None
697,None,None,None
698,None,None,None


In [118]:
listing_info = {"Price":data["Price"], "Beds":data["Beds"], "Ammenities":data["Ammenities"]}
initial_data1 = pd.DataFrame(listing_info).iloc[0:128]


In [119]:
identify_listing = {"Title":data["Title"], "Address":data["Title"]}
initial_data2 = pd.DataFrame(identify_listing)



In [122]:
sample_data = initial_data2.join(initial_data1)

In [125]:
sample_data

,Title,Address,Price,Beds,Ammenities
0,The Landmark Los Angeles,The Landmark Los Angeles,"$3,880 - $11,750",Studio - 2 Beds,Pets Allowed Fitness Center Pool In Unit Washe...
1,The Amara,The Amara,"$3,240 - $4,955",1-2 Beds,Pets Allowed Fitness Center Pool Refrigerator ...
2,Ten Thousand,Ten Thousand,"$10,915 - $65,000",1-4 Beds,Pets Allowed Fitness Center Pool Dishwasher Re...
3,2245 Beverly Glen Blvd.,2245 Beverly Glen Blvd.,"$2,298 - $3,348",Studio - 2 Beds,None
4,Linea,Linea,"$3,145 - $9,200",Studio - 3 Beds,Pets Allowed Fitness Center Pool Dishwasher Re...
...,...,...,...,...,...
123,2040 S Beverly Glen Blvd Unit # 4,2040 S Beverly Glen Blvd Unit # 4,"$1,900","1 Bed, 1 Bath, 500 sq ft",None
124,1813 Barry Ave,1813 Barry Ave,"$5,250","2 Beds, 2.5 Baths, 1,700 sq ft",None
125,1437 Brockton Ave Unit 05,1437 Brockton Ave Unit 05,"$2,850","2 Beds, 1 Bath, 1,000 sq ft",None
126,12124 Goshen Ave Unit 106,12124 Goshen Ave Unit 106,"$4,800","2 Beds, 2.5 Baths, 1,750 sq ft",None


## Scratch work

In [78]:
while(True):
    time.sleep(2)
    button_list = driver.find_elements(By.CLASS_NAME, "pagingBtn")
    next = button_list[len(button_list) - 1]
    if next.text == "Next":
        next.click()
    else:
        break

In [ ]:
title, address, price, beds, ammenities = [], [], [], [], []
title_addr = driver.find_elements(By.CLASS_NAME, "property-information")
room_info = driver.find_elements(By.CLASS_NAME, "property-info") # aggregate listing info 

for listing in title_addr:
    title.append(listing.find_element(By.CLASS_NAME, "property-title").text)
    address.append(listing.find_element(By.CLASS_NAME, "property-address").text)

for listing in room_info:
    price.append(listing.find_element(By.CLASS_NAME, "property-pricing").text)
    beds.append(listing.find_element(By.CLASS_NAME, "property-beds").text)
    try:
        ammenities.append(listing.find_element(By.CLASS_NAME, "property-amenities").text)
    except:
        ammenities.append(None)


,Title,Address,Price,Beds,Ammenities
37,Veteran,"1417 Veteran Ave, Los Angeles, CA 90024","$2,595",1 Bed,None
28,Kith + Kin,"11977 Kiowa Ave, Los Angeles, CA 90049","$4,962 - $5,856",3 Beds,Pets Allowed Dishwasher
31,Picasso Brentwood,"12027-12035 Wilshire Blvd, Los Angeles, CA 90025","$3,195 - $4,400",Studio - 3 Beds,Fitness Center Pool Dishwasher
16,The Plaza Apartments,"10983-10983 Wellworth Ave, Los Angeles, CA 90024","$2,575 - $4,318",1-2 Beds,Cats Allowed Fitness Center Dishwasher
30,2225 Beverly Glen Blvd.,"2225 S Beverly Glen Blvd, Los Angeles, CA 90064","$2,998",2 Beds,Dishwasher Refrigerator Walk-In Closets


In [46]:
driver.close()

In [ ]:
location_info = driver.find_elements(By.CLASS_NAME, "property-information")
location_info[0].find_element(By.CLASS_NAME, "property-title").text # Source title 
location_info[0].find_element(By.CLASS_NAME, "property-address").text # source address


'11750 Wilshire Blvd, Los Angeles, CA 90025'

In [32]:
room_info = driver.find_elements(By.CLASS_NAME, "property-info") # aggregate listing info 


In [ ]:
room_info[0].find_element(By.CLASS_NAME, "property-pricing").text # Source pricing 
"property-beds" # Source bedrooms

'$3,880 - $11,750'

In [ ]:
room_info[0].find_element(By.CLASS_NAME, "property-amenities").text # list some of the amenities 

'Pets Allowed Fitness Center Pool In Unit Washer & Dryer'